In [1]:
import os
import numpy as np
import pandas as pd
import shutil

In [2]:
def mk_dir(dirpath):
    if not os.path.exists(dirpath):
        os.mkdir(dirpath)

In [3]:
dirpath = os.getcwd()
lbl_dir = os.path.join(os.path.normpath(dirpath + os.sep + os.pardir), r"udder_labels\labels\keypoints")
label_list = [file.replace(".txt", "") for file in os.listdir(lbl_dir)]
ped_dir = r"validate_watershed\pred_labels\keypoints"
pred_list = [file.replace(".txt", "") for file in os.listdir(ped_dir)]
group_dir = r"validate_watershed\survey_groups"
img_path = r"validate_watershed\watershed_examples"

img_list = sorted(list(set(pred_list).difference(set(label_list))))
cow_list = [file.split("_")[0] for file in img_list]
img_df = pd.DataFrame({"cow": cow_list, "filename": img_list})
cow_list2, counts = np.unique(cow_list, return_counts=True)
cow_list2 = cow_list2[np.argsort(counts)]
ngroups = 30
img_pgroup = 10
total_images = ngroups*img_pgroup
total_cows = len(cow_list2)
img_pcow = total_images//total_cows
cows_wex = total_images%total_cows
img_pcow_list = [img_pcow]*(total_cows - cows_wex) + [img_pcow +1 ] *cows_wex
gorup_list = list(range(ngroups))*img_pgroup
group_idx = np.repeat(list(range(img_pgroup)), ngroups)

In [4]:
selframes_df = pd.DataFrame(columns = ["cow", "filename", "group", "idx"])
frame_dict = {}
for i, cow in enumerate(cow_list2):
    cow_im = img_pcow_list[i]
    cow_frames = list(img_df[img_df.cow == cow]["filename"])
    np.random.seed(5)
    np.random.shuffle(cow_frames)
    selected_frames = cow_frames[:cow_im]
    temp_df = pd.DataFrame({"cow": [cow]*cow_im, "filename": selected_frames, "group": None} )
    selframes_df = pd.concat([selframes_df, temp_df], axis = 0, ignore_index = True)
    frame_dict[cow] = selected_frames
    
selframes_df["group"] = gorup_list
selframes_df["idx"] = group_idx

for i  in range(len(selframes_df)):
    line = selframes_df.loc[i]
    filename = line.filename + ".png"
    out_name = "g" + str(line.group) + "_" + str(line.idx) +".png"
    src = os.path.join(img_path, line.cow, filename)
    out_path = os.path.join(group_dir, "group_" + str(line.group))
    mk_dir(out_path)
    dest =  os.path.join(out_path, out_name)
    shutil.copy(src, dest)

In [15]:
selframes_df.to_csv("validate_watershed\survey_grous.csv", index= False)